In [8]:
from td_lab.models import MaternalRequisition
from td_maternal.models import MaternalVisit, RapidTestResult, AntenatalEnrollment

In [9]:
maternal_requisition = MaternalRequisition.objects.all()

In [10]:
rs_fields = [
    'subject_identifier', 'screening_identifier', 'dob', 'gender', 'subject_type', 'sid',
    'screening_age_in_years', 'registration_status', 'registration_datetime', 'randomization_datetime']
visit_fields = [
    'visit_datetime', 'study_status', 'reason', 'survival_status', 'last_alive_date', 'maternal_visit_id']
visit_defination_fields = ['visit_code', 'visit_title', 'time_point']
appointment_fields = ['appt_status', 'appt_datetime', 'timepoint_datetime', 'best_appt_datetime']
rando_fields = ['rx']
enrollement_fields = ['enrollment_hiv_status']
rapid_test_fields = ['rapid_test_done', 'rapid_test_date', 'result']

In [11]:
requisition_fields = ['created',
 'modified',
 'user_created',
 'user_modified',
 'hostname_created',
 'hostname_modified',
 'revision',
 'id',
 'report_datetime',
 'exported',
 'exported_datetime',
 'export_change_type',
 'export_uuid',
 'subject_identifier',
 'requisition_identifier',
 'requisition_datetime',
 'specimen_identifier',
 'protocol',
 'study_site',
 'clinician_initials',
 'priority',
 'is_drawn',
 'reason_not_drawn',
 'drawn_datetime',
 'item_type',
 'item_count_total',
 'estimated_volume',
 'comments',
 'is_receive',
 'is_receive_datetime',
 'is_packed',
 'is_labelled',
 'is_labelled_datetime',
 'is_lis',
 'packing_list_id',
 'aliquot_type_id',
 'panel_name',
 'reason_not_drawn_other',
 'test_code']
header = rs_fields + visit_fields + visit_defination_fields + appointment_fields + rando_fields + enrollement_fields + rapid_test_fields + requisition_fields
data = [header]
for m_requisition in maternal_requisition:
    visit = m_requisition.maternal_visit
    appointment = visit.appointment
    rs = appointment.registered_subject
    visit_defination = appointment.visit_definition

    # Registered_subject data
    rs_data = []
    for rs_f in rs_fields:
        rs_data.append(getattr(rs, rs_f))

    # Visit data
    v_data = []
    for v_f in visit_fields:
        if v_f == 'maternal_visit_id':
            v_data.append(getattr(visit, 'id'))
        elif v_f == 'visit_datetime':
            v_data.append(getattr(visit, 'report_datetime'))
        else:
            v_data.append(getattr(visit, v_f))

    # Visit defination data
    vdef_data = []
    for vdef_f in visit_defination_fields:
        if vdef_f == 'visit_code':
            vdef_data.append(getattr(visit_defination, 'code'))
        elif vdef_f == 'visit_title':
            vdef_data.append(getattr(visit_defination, 'title'))
        else:
            vdef_data.append(getattr(visit_defination, vdef_f))

    # Appointment data
    app_data = []
    for ap_f in appointment_fields:
        app_data.append(getattr(appointment, ap_f))

    # Rando data
    rando_data = []
    try:
        rando = MaternalRando.objects.get(subject_identifier=visit.appointment.registered_subject.subject_identifier)
    except MaternalRando.DoesNotExist:
        rando_data.append('.')
    else:
        rando_data.append(getattr(rando, 'rx'))


    # Enrollemnt data
    enrollment_data = []
    try:
        enrollment = AntenatalEnrollment.objects.get(registered_subject=rs)
    except AntenatalEnrollment.DoesNotExist:
        print('there is a problem')
    else:
        enrollment_data.append(getattr(enrollment, 'enrollment_hiv_status'))


    # Rapied Test data
    rapied_test_data = []
    try:
        rapied_test = RapidTestResult.objects.get(maternal_visit=visit)
    except RapidTestResult.DoesNotExist:
        rapid_test_done = 'No'
        rapid_test_date = '.'
        result = '.'
        rapied_test_data = [rapid_test_done, rapid_test_date, result]
    else:
        rapied_test_data = [rapied_test.rapid_test_done, rapied_test.result_date, rapied_test.result]


    # Requisition data
    requisition_data = []
    for req_f in requisition_fields:
        if req_f == 'panel_name':
            requisition_data.append(m_requisition.panel.name)
        else:
            value = getattr(m_requisition, req_f)
            if not value:
                value = '.'
            requisition_data.append(value)
    all_data = rs_data + v_data + vdef_data + app_data + rando_data + enrollment_data + rapied_test_data + requisition_data
    data.append(all_data)

In [12]:
print(len(data))
import csv
with open('td_lab_maternalrequisition.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerows(data)

2855
